In [1]:
import cv2
import mediapipe as mp
import numpy as np

### Task 1 - Object Detection (Hands)

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

try:
    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("No frame received, exiting...")
                break

            # mirror effect
            frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # hand detection
            results = hands.process(rgb_frame)

            # landmarks and boxes
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=0),
                        mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=0)
                    )

                    # box calculations
                    h, w, _ = frame.shape
                    x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * w)
                    y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * h)
                    x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * w)
                    y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * h)

                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

            cv2.imshow('Hand Detection', frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('e'):
                break

            # detect if window is closed
            if cv2.getWindowProperty('Hand Detection', cv2.WND_PROP_VISIBLE) < 1:
                break

except Exception as e:
    print(f"Error: {e}")

finally:
    cap.release()
    cv2.destroyAllWindows()

### Task 2 - Video Skeleton

In [11]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [15]:
video_path = "media/rdance1.mp4"
cap = cv2.VideoCapture(video_path)

In [16]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = pose.process(rgb_frame)

        # Create a side black background
        skeleton_frame = np.zeros_like(frame)

        # Draw the skeleton on both the original frame and the black background
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2), 
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)   
            )
            mp_drawing.draw_landmarks(
                skeleton_frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2), 
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)   
            )

        # Combine the two frames horizontally
        combined_frame = cv2.hconcat([frame, skeleton_frame])

        # Display the combined frame
        cv2.imshow('Skeleton and Original', combined_frame)

        if cv2.waitKey(1) & 0xFF == ord('e'):
            break

cap.release()
cv2.destroyAllWindows()